# Using EcoFOCIpy to process raw field data

## DY1307 ( - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [22]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [23]:
sample_data_dir = '/Users/bell/ecoraid/2013/CTDcasts/dy1307l2/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [24]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY1307L2' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY1307L2.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDwWetLabs.yaml'
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd017.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd018.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd021.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd024.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd027.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1307l2/rawconverted/ctd030.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [26]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd017.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,t090c,t190c,prdm,wetstar,scan,datetime
count,1.0000,1.00,1.000,1.0000,1.0000,1.0000,1.0000,1.0000,1.000,1.000,1.0000,1.0000,1.000,1.0000,1.0,1
mean,5.4435,74.63,237.021,32.4026,32.4007,25.6758,25.6745,5.1803,71.022,225.562,4.4433,4.4415,173.147,0.0857,10897.0,2013-07-01 10:50:00
min,5.4435,74.63,237.021,32.4026,32.4007,25.6758,25.6745,5.1803,71.022,225.562,4.4433,4.4415,173.147,0.0857,10897.0,2013-07-01 10:50:00
25%,5.4435,74.63,237.021,32.4026,32.4007,25.6758,25.6745,5.1803,71.022,225.562,4.4433,4.4415,173.147,0.0857,10897.0,2013-07-01 10:50:00
50%,5.4435,74.63,237.021,32.4026,32.4007,25.6758,25.6745,5.1803,71.022,225.562,4.4433,4.4415,173.147,0.0857,10897.0,2013-07-01 10:50:00
75%,5.4435,74.63,237.021,32.4026,32.4007,25.6758,25.6745,5.1803,71.022,225.562,4.4433,4.4415,173.147,0.0857,10897.0,2013-07-01 10:50:00
max,5.4435,74.63,237.021,32.4026,32.4007,25.6758,25.6745,5.1803,71.022,225.562,4.4433,4.4415,173.147,0.0857,10897.0,2013-07-01 10:50:00
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create BTL report file

In [27]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd017':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [28]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY1307L2',
 'CruiseID_Historic': 'DY1307',
 'CruiseID_Alternates': '',
 'Project_Leg': 'L2',
 'Vessel': 'R/V Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2013, 6, 30),
 'EndDate': datetime.date(2013, 7, 18),
 'Project': 'MACE',
 'ChiefScientist': 'Neal Williamson',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Kodiak, AK',
 'CruiseLocation': '',
 'Description': 'Gulf of Alaska Acoustic-Trawl Survey of Walleye Pollock, CTD',
 'CruiseYear': 2013,
 'ctdlogs_pdf_name': 'DY1307_CTDLogs.pdf'}

In [29]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD017']

{'id': 21690,
 'Vessel': 'R/V Oscar Dyson',
 'CruiseID': 'DY1307',
 'Project_Leg': 'L2',
 'UniqueCruiseID': 'DY1307L2',
 'Project': '',
 'StationNo_altname': '17',
 'ConsecutiveCastNo': 'CTD017',
 'LatitudeDeg': 58,
 'LatitudeMin': 42.33,
 'LongitudeDeg': 152,
 'LongitudeMin': 43.93,
 'GMTDay': 1,
 'GMTMonth': 'Jul',
 'GMTYear': 2013,
 'GMTTime': 39000,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': 1006,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': 122,
 'WindSpd': 18.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 181,
 'StationNameID': 'ME70',
 'MaxDepth': 170,
 'InstrumentSerialNos': 'Press SN = 0772, Pri Temp SN = 2376, Sec Temp SN = 4379, Pri Cond SN = 2985, Sec Cond Sn = 3127, PAR Sn = 70281, Fluor Sn = WETStar 868, pri O2 Sn = 0904, sec O2 Sn = 1961',
 'Notes': '',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNis

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [30]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [33]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd017.btl'] = cruise_data['ctd017.btl'].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

cruise_data['ctd017.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,sbeox1ml/l,sbeox1ps,sbox1mm/kg,temperature_ch1,temperature_ch2,pressure,wetstar,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,
1.0,5.4435,74.63,237.021,32.4026,32.4007,25.6758,25.6745,5.1803,71.022,225.562,4.4433,4.4415,173.147,0.0857,10897.0,2013-07-01 10:50:00,ctd017


In [34]:
cruise_data['ctd017.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'sbeox1ml/l', 'sbeox1ps',
       'sbox1mm/kg', 'temperature_ch1', 'temperature_ch2', 'pressure',
       'wetstar', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [35]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [37]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd017.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [38]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 1)
Coordinates:
  * bottle              (bottle) float64 1.0
Data variables: (12/17)
    oxy_concM_ch1       (bottle) float64 5.444
    oxy_percentsat_ch1  (bottle) float64 74.63
    oxy_conc_ch1        (bottle) float64 237.0
    salinity_ch1        (bottle) float64 32.4
    salinity_ch2        (bottle) float64 32.4
    sigma_t_ch1         (bottle) float64 25.68
    ...                  ...
    temperature_ch2     (bottle) float64 4.441
    pressure            (bottle) float64 173.1
    wetstar             (bottle) float64 0.0857
    scan                (bottle) float64 1.09e+04
    datetime            (bottle) datetime64[ns] 2013-07-01T10:50:00
    cast                (bottle) object 'ctd017'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [39]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [41]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd017.btl'].columns.values),drop_missing=True)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [42]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 1, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables:
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 5.444
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 74.63
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 237.0
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.4
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.4
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.68
    sigma_t_ch2         (latitude, longitude, time, bottle) float64 25.67
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.443
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.441
    pressure            (latitude, longitude, time, bottle) float64 173.1

In [44]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD017')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 1, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables:
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 5.444
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 74.63
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 237.0
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.4
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.4
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.68
    sigma_t_ch2         (latitude, longitude, time, bottle) float64 25.67
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.443
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.441
    pressure            (latitude, longitude, time, bottle) float64 173.1
Attributes: (12/44)
    CruiseID:                      DY1307L2
    CruiseID_Historic:             DY1307
    CruiseID_Alternates:           
    Project_Leg:                   L2
    Vessel:                        R/V Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2023-09-08T14:53:29Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [45]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history